In [6]:
import numpy as np
import torch
from go.GoGame import GoGame
from go.NNet import NNetWrapper as NNet
from MCTS import MCTS
from Config import get_config

def main():
    IterNumber = 45
    seed = IterNumber
    parser = get_config(seed)
    args = parser.parse_args([])

    game = GoGame(args.boardsize, "tic-tac-toe") 

    nnet = NNet(args.boardsize, args)
    nnet.load_checkpoint('./temp/Iter31/', 'best.pth.tar')

    mcts = MCTS(nnet, args.policy, args)

    board = game.board
    curPlayer = game.cur_player

    while True:
        print(f"Player {curPlayer} to move:")
        game.display()

        ended = game.getGameEnded()
        if ended != -1:
            print("Game Over!")
            if ended == 1:
                print("Player 1 wins!")
            elif ended == 0:
                print("Player -1 wins!")
            else:
                print("Draw!")
            break

        action_probs = mcts.getActionProb(game)
        action = np.argmax(action_probs)

        game.ExcuteAction(action)
        board = game.board
        curPlayer = game.cur_player

if __name__ == "__main__":
    main()


Player 1 to move:
0 |1 |2 |
 -----------------------
0 |- - - |
1 |- - - |
2 |- - - |
   -----------------------


AttributeError: 'GoGame' object has no attribute 'win_num'

In [17]:
from copy import deepcopy
import torch
import numpy as np
import random
import time
from go.GoGame import GoGame
from go.NNet import NNetWrapper as nn
from MCTS import MCTS  # 你自己的 MCTS 文件
from argparse import Namespace

# 将 1~9 转换为 board 上的 (x,y)
def action_to_xy(action):
    return divmod(action, 3)  # (row, col)

def print_board(board):
    symbols = {0: ' ', 1: 'X', -1: 'O'}
    print("\n  1 2 3\n -------")
    for i in range(3):
        row = f"{i+1}|"
        for j in range(3):
            row += symbols[int(board[i][j])] + " "
        print(row)
    print(" -------")

def get_user_action(valids):
    while True:
        try:
            pos = int(input("请输入你的落子位置（1-9）：")) - 1
            if pos < 0 or pos >= 9 or valids[pos] == 0:
                print("无效位置，请重新输入！")
            else:
                return pos
        except:
            print("输入错误，请输入数字1~9")

def main():
    # 配置参数（如果你有 config parser，可以替代这个）
    IterNumber = int(45)
    
    seed = IterNumber
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    parser = get_config(IterNumber)
    args = parser.parse_args([])

    # 初始化棋盘与神经网络
    game = GoGame(3, "tic_tac_toe")
    nnet = nn(3, args)
    nnet.load_checkpoint('./.checkpoints/', 'uct_45.pth.tar')

    mcts = MCTS(nnet, game, args)

    board = game.board
    player = 1  # 玩家 1 先手（X）

    print("欢迎来到井字棋人机对战！你是 X（先手），AI 是 O（后手）")
    print_board(board)

    while True:
        valids = game.getValidMoves()
        game_result = game.getGameEnded()

        if game_result != -1:
            print_board(game.board)
            if game_result == 1:
                print("你赢了！")
            elif game_result == 0:
                print("你输了！")
            else:
                print("平局！")
            break

        if game.cur_player == 1:
            action = get_user_action(valids)
        else:
            print("AI 正在思考中...")
            pi = mcts.getActionProb(deepcopy(game))
            action = np.argmax(pi)
            time.sleep(0.5)

        game.ExcuteAction(action)
        print_board(game.board)
        
if __name__ == "__main__":
    main()


欢迎来到井字棋人机对战！你是 X（先手），AI 是 O（后手）

  1 2 3
 -------
1|      
2|      
3|      
 -------



  1 2 3
 -------
1|    X 
2|      
3|      
 -------
AI 正在思考中...

  1 2 3
 -------
1|    X 
2|  O   
3|      
 -------

  1 2 3
 -------
1|  X X 
2|  O   
3|      
 -------
AI 正在思考中...

  1 2 3
 -------
1|O X X 
2|  O   
3|      
 -------

  1 2 3
 -------
1|O X X 
2|  O   
3|    X 
 -------
AI 正在思考中...

  1 2 3
 -------
1|O X X 
2|  O   
3|  O X 
 -------

  1 2 3
 -------
1|O X X 
2|X O   
3|  O X 
 -------
AI 正在思考中...

  1 2 3
 -------
1|O X X 
2|X O   
3|O O X 
 -------

  1 2 3
 -------
1|O X X 
2|X O X 
3|O O X 
 -------

  1 2 3
 -------
1|O X X 
2|X O X 
3|O O X 
 -------
平局！
